# **Supplementary Code 4**
This notebook was used for analysis of NGS reads containing intended prime-editing and synonymous mutation marker. For more detail, please read Methods and Supplementary Information. 

Lead contact: Hyoungbum Henry Kim (hkim1@gmail.com)

Technical contact: Goosang Yu (gsyu93@gmail.com), Yusang Jung (ys.jung@yuhs.ac)

## Directory tree

📦Working directory  
 ┣ 📂data  
 ┃ ┣ 📂NGS_FASTQ_files  
 ┃ ┣ 📂NGS_frequency_table  
 ┃ ┃ ┣ 📜C4Bosutinib791.txt  
 ┃ ┃ ┣ 📜C4Control797.txt  
 ┃ ┃ ┗ 📜...  
 ┃ ┣ 📂read_counts  
 ┃ ┣ 📂statistics  
 ┃  
 ┣ 📂src  
 ┃ ┣ 📜Alignment.py  
 ┃ ┣ 📜VarCalling.py  
 ┃  
 ┣ 📂variants_info  
 ┃ ┣ 📜ex4_info.csv  
 ┃ ┣ 📜ex5_info.csv  
 ┃ ┣ 📜ex6_info.csv  
 ┃ ┣ 📜ex7_info.csv  
 ┃ ┣ 📜ex8_info.csv  
 ┃ ┣ 📜ex9_info.csv  
 ┃ ┣ 📜invivo_ex4_info.csv  
 ┃ ┗ 📜invivo_ex9_info.csv  
 ┃  
 ┗ 📜SuppleCode4.ipynb (this file)  

# Requirements
- CRISPResso2 (>= 2.x.x)
- pandas

## Variants calling and make read count file
After running CRISPResso, generate the read count file. This is the process of creating the foundational file for all analyses.

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from glob import glob

from src.Alignment import ABL1VUS
from src.VarCalling import make_count_file, read_statistics, combine_data, VariantFilter, VariantScore, Normalizer

In [ ]:
# Make count files from frequency table

freq_table = 'frequency_table/KCLPE4K_DoseControl_Exon5_Rep1_DMSO.txt'
ref_info   = 'variants_info/ex5_info.csv'

df_cnt = make_count_file(freq_table, ref_info)

In [2]:
# Make count files from frequency table

freq_tables = glob('data/frequency_table_ex4_day0/*.txt')

for f in freq_tables:

    n_sample = os.path.basename(f).replace('.txt', '')
    exon_num = n_sample.split('Exon')[1][0]
    ref_info = f'variants_info/ex{exon_num}_info.csv'

    
    df_cnt = make_count_file(f, ref_info)
    df_cnt.to_csv(f'data/read_counts/Count_{n_sample}.csv', index=False)

[Info] Read counting: K562PE4K_HTS_Exon4_day0: 100%|██████████| 1282602/1282602 [02:05<00:00, 10213.19it/s]


In [3]:
# Make count files from frequency table

freq_tables = glob('data/frequency_table/*.txt')

for f in freq_tables:

    n_sample = os.path.basename(f).replace('.txt', '')
    exon_num = n_sample.split('Exon')[1][0]
    ref_info = f'variants_info/ex{exon_num}_info.csv'

    
    df_cnt = make_count_file(f, ref_info)
    df_cnt.to_csv(f'data/read_counts/Count_{n_sample}.csv', index=False)

[Info] Read counting: K562PE2_unedit_Exon5: 100%|██████████| 36438/36438 [00:03<00:00, 10306.87it/s]
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_Asciminib: 100%|██████████| 1126274/1126274 [01:52<00:00, 9996.74it/s] 
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_Bosutinib: 100%|██████████| 1310545/1310545 [02:08<00:00, 10160.25it/s]
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_Dasatinib: 100%|██████████| 1096266/1096266 [01:48<00:00, 10090.48it/s]
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_DMSO: 100%|██████████| 1231441/1231441 [02:02<00:00, 10064.29it/s]
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_Imatinib: 100%|██████████| 1290582/1290582 [02:08<00:00, 10016.11it/s]
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_Nilotinib: 100%|██████████| 1059960/1059960 [01:47<00:00, 9905.26it/s] 
[Info] Read counting: K562PE4K_HTS_Exon4_Rep1_Ponatinib: 100%|██████████| 964463/964463 [01:36<00:00, 9955.14it/s] 
[Info] Read counting: K562PE4K_HTS_Exon4_Rep2_Asciminib: 100%|██████████| 1532951

In [1]:
import pandas as pd

df = pd.read_csv('data/statistics/Stat_KCLPE4K_DoseControl_Exon5_Rep2_DMSO.csv')
df

,RefSeq,Label,AA_var,SNV_var,count,frequency,Edited_WT_count,RPM,UE_SynPE_count,UE_WT_count,OR,pvalue
0,TTCCTTCTGCAGAAGGACACTATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,E275K,exon5_pos1G_A,63516,0.006535,3021820,16039.102319,1755,20941910,250.676164,0.000000e+00
1,TTCCTTCTGCAGAAGGACACCATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,E275K,exon5_pos1G_A,1117,0.000115,3021820,282.065579,3644,20941910,2.125650,1.771596e-93
2,TTCCTTCTGCAGCAGGACACTATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,E275Q,exon5_pos1G_C,35858,0.003689,3021820,9054.885871,2030,20941910,122.358974,0.000000e+00
3,TTCCTTCTGCAGCAAGACACCATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,E275Q,exon5_pos1G_C,49,0.000005,3021820,12.373512,0,20941910,346.511441,8.612879e-45
4,TTCCTTCTGCAGTAGGACACTATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,E275Stop,exon5_pos1G_T,582,0.000060,3021820,146.967025,86,20941910,46.440499,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
491,TTCCTTCTGCAGGAGGACACCATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,G303R,exon5_pos85G_A,138,0.000014,3021820,34.847851,22,20941910,41.882687,2.538697e-99
492,TTCCTTCTGCAGGAGGACACCATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,G303R,exon5_pos85G_C,415,0.000043,3021820,104.796074,34,20941910,82.370720,0.000000e+00
493,TTCCTTCTGCAGGAGGACACCATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,G303R,exon5_pos85G_C,307,0.000032,3021820,77.523843,28,20941910,73.603810,1.011078e-237
494,TTCCTTCTGCAGGAGGACACCATGGAGGTGGAAGAGTTCTTGAAAG...,SynPE,G303W,exon5_pos85G_T,612,0.000063,3021820,154.542645,67,20941910,62.473975,0.000000e+00


## Significance analysis
Compare the prime-edited sample with the unedited sample to calculate the odds ratio and Fisher's exact test p-value for each variant.

In [3]:
# Calculate statistics with read count for each variants

list_control = [
    'K562PE4K_HTS_Exon4_Rep1_DMSO',
    'K562PE4K_HTS_Exon4_Rep2_DMSO',
    'K562PE4K_HTS_Exon5_Rep1_DMSO',
    'K562PE4K_HTS_Exon5_Rep2_DMSO',
    'K562PE4K_HTS_Exon6_Rep1_DMSO',
    'K562PE4K_HTS_Exon6_Rep2_DMSO',
    'K562PE4K_HTS_Exon7_Rep1_DMSO',
    'K562PE4K_HTS_Exon7_Rep2_DMSO',
    'K562PE4K_HTS_Exon8_Rep1_DMSO',
    'K562PE4K_HTS_Exon8_Rep2_DMSO',
    'K562PE4K_HTS_Exon9_Rep1_DMSO',
    'K562PE4K_HTS_Exon9_Rep2_DMSO',
    ]

for test_sample in list_control:
    
    background_sample = test_sample.split('_Rep')[0].replace('HTS', 'unedit')

    test_file       = f'data/read_counts/Count_{test_sample}.csv'
    background_file = f'data/read_counts/Count_{background_sample}.csv'

    df_stats = read_statistics(test_file, background_file)

    df_stats.to_csv(f'data/statistics/Stat_{test_sample}.csv', index=False)

Analysis: Count_K562PE4K_HTS_Exon4_Rep1_DMSO
Analysis: Count_K562PE4K_HTS_Exon4_Rep2_DMSO
Analysis: Count_K562PE4K_HTS_Exon5_Rep1_DMSO
Analysis: Count_K562PE4K_HTS_Exon5_Rep2_DMSO
Analysis: Count_K562PE4K_HTS_Exon6_Rep1_DMSO
Analysis: Count_K562PE4K_HTS_Exon6_Rep2_DMSO
Analysis: Count_K562PE4K_HTS_Exon7_Rep1_DMSO
Analysis: Count_K562PE4K_HTS_Exon7_Rep2_DMSO
Analysis: Count_K562PE4K_HTS_Exon8_Rep1_DMSO
Analysis: Count_K562PE4K_HTS_Exon8_Rep2_DMSO
Analysis: Count_K562PE4K_HTS_Exon9_Rep1_DMSO
Analysis: Count_K562PE4K_HTS_Exon9_Rep2_DMSO


## Day 0 vs Day 10 (DMSO treated) analysis
Analysis to verify proliferation effects of synonymous/nonsense mutations before assessing drug resistance

- Test: After creating variants using prime editing for 20 days, treat with DMSO (negative TKI control) for 10 days.
- Control: After creating variants using prime editing for 20 days, immediately harvest.

In [2]:
# Filtering variants and calculate normalized log 2 fold changes
sample_tag = 'K562PE4K_HTS'

list_sample = [
    f'{sample_tag}_Exon4',
    f'{sample_tag}_Exon5',
    f'{sample_tag}_Exon6',
    f'{sample_tag}_Exon7',
    f'{sample_tag}_Exon8',
    f'{sample_tag}_Exon9',
    ]

list_chem = [
    'Imatinib', 
    'Nilotinib', 
    'Bosutinib', 
    'Dasatinib', 
    'Ponatinib', 
    'Asciminib', 
]

for sample in list_sample:
    '''Day 0 sample은 replicate 1/2가 없는 것들이 있음. 
    Exon4 - Rep2
    Exon5 - Rep2
    Exon6 - Rep1 / 2
    Exon7 - Rep1
    Exon8 - Rep2
    Exon9 - Rep1 / 2
    '''
    print(f'Filtering and Normalization: {sample}')
    test_r1 = f'data/read_counts/Count_{sample}_Rep1_day0.csv'
    test_r2 = f'data/read_counts/Count_{sample}_Rep2_day0.csv'

    control_r1 = f'data/statistics/Stat_{sample}_Rep1_DMSO.csv'
    control_r2 = f'data/statistics/Stat_{sample}_Rep2_DMSO.csv'
    
    df_rep1, df_rep2 = VariantFilter(test_r1, test_r2, control_r1, control_r2).filter(OR_cutoff=2, p_cutoff=0.05, p_column='adj_pvalue', rpm_cutoff=10)

    normal = Normalizer()

    for lws_frac in [0.15, 0.2, 0.3, 0.4]:

        # LOWESS regression normalization: Calculate log fold change by swapping Day0 as control and DMSO as test.
        df_nor1 = normal.lowess(df_rep1, frac=lws_frac, control='test', test='control')
        df_nor2 = normal.lowess(df_rep2, frac=lws_frac, control='test', test='control')

        df_nor1.to_csv(f'data/statistics/Filtered_{lws_frac}_{sample}_Rep1_Day0DMSO.csv', index=False)
        df_nor2.to_csv(f'data/statistics/Filtered_{lws_frac}_{sample}_Rep2_Day0DMSO.csv', index=False)
    
    # Z-score normalization: Day0 = control, DMSO = test로 뒤바꿔서 log fold change를 계산
    df_nor1 = normal.zscore(df_rep1, control='test', test='control')
    df_nor2 = normal.zscore(df_rep2, control='test', test='control')

    df_nor1.to_csv(f'data/statistics/Filtered_zscore_{sample}_Rep1_Day0DMSO.csv', index=False)
    df_nor2.to_csv(f'data/statistics/Filtered_zscore_{sample}_Rep2_Day0DMSO.csv', index=False)


for nor_option in [0.15, 0.2, 0.3, 0.4, 'zscore']:

    # Exon combine
    for rep in [1, 2]:
        print(f'Combine: {sample_tag}-Day0_vs_DMSO-Replicate{rep}-Normalization {nor_option}')
        files = glob(f'data/statistics/Filtered_{nor_option}_{sample_tag}_Exon*_Rep{rep}_Day0DMSO.csv')
        df_merged = combine_data(files)
        df_merged.to_csv(f'data/statistics/Filtered_{nor_option}_{sample_tag}_AllExons_Rep{rep}_Day0DMSO.csv', index=False)


    # Score calculation: Adjusted LFC / Resistance score
    score = VariantScore()

    rep_1 = f'data/statistics/Filtered_{nor_option}_{sample_tag}_AllExons_Rep1_Day0DMSO.csv'
    rep_2 = f'data/statistics/Filtered_{nor_option}_{sample_tag}_AllExons_Rep2_Day0DMSO.csv'

    adjus_LFC = score.calculate(rep_1, rep_2, var_type='SNV')
    res_score = score.calculate(rep_1, rep_2, var_type='AA')

    adjus_LFC.to_csv(f'data/adjusted_LFC/AdjustedLFC_{nor_option}_{sample_tag}_Day0DMSO.csv')
    res_score.to_csv(f'data/resistance_score/ResistanceScore_{nor_option}_{sample_tag}_Day0DMSO.csv')



Filtering and Normalization: K562PE4K_HTS_Exon4
Filtering and Normalization: K562PE4K_HTS_Exon5
Filtering and Normalization: K562PE4K_HTS_Exon6
Filtering and Normalization: K562PE4K_HTS_Exon7
Filtering and Normalization: K562PE4K_HTS_Exon8
Filtering and Normalization: K562PE4K_HTS_Exon9
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate1-Normalization 0.15
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate2-Normalization 0.15
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate1-Normalization 0.2
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate2-Normalization 0.2
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate1-Normalization 0.3
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate2-Normalization 0.3
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate1-Normalization 0.4
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate2-Normalization 0.4
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate1-Normalization zscore
Combine: K562PE4K_HTS-Day0_vs_DMSO-Replicate2-Normalization zscore


## DMSO vs TKI response analysis
Analysis for resistance to drugs

- Test: Making variants using Prime editing for 20 days, followed by 10 days of TKI treatment.
- Control: Making variants using Prime editing for 20 days, followed by 10 days of DMSO treatment.

In [3]:
# Filtering variants and calculate normalized log 2 fold changes
list_sample = [
    'K562PE4K_HTS_Exon4',
    'K562PE4K_HTS_Exon5',
    'K562PE4K_HTS_Exon6',
    'K562PE4K_HTS_Exon7',
    'K562PE4K_HTS_Exon8',
    'K562PE4K_HTS_Exon9',
    ]

list_chem = [
    'Imatinib', 
    'Nilotinib', 
    'Bosutinib', 
    'Dasatinib', 
    'Ponatinib', 
    'Asciminib', 
]

lws_frac = 0.15

for sample in list_sample:
    for chem in list_chem:

        test_r1 = f'data/read_counts/Count_{sample}_Rep1_{chem}.csv'
        test_r2 = f'data/read_counts/Count_{sample}_Rep2_{chem}.csv'

        control_r1 = f'data/statistics/Stat_{sample}_Rep1_DMSO.csv'
        control_r2 = f'data/statistics/Stat_{sample}_Rep2_DMSO.csv'
        
        df_rep1, df_rep2 = VariantFilter(test_r1, test_r2, control_r1, control_r2).filter(OR_cutoff=2, p_cutoff=0.05, p_column='adj_pvalue', rpm_cutoff=10)

        normal = Normalizer()

        # LOWESS regression normalization
        df_nor1 = normal.lowess(df_rep1, frac=lws_frac)
        df_nor2 = normal.lowess(df_rep2, frac=lws_frac)

        df_nor1.to_csv(f'data/statistics/Filtered_{sample}_Rep1_{chem}.csv', index=False)
        df_nor2.to_csv(f'data/statistics/Filtered_{sample}_Rep2_{chem}.csv', index=False)

### Combine all exons data

In [4]:
# Exon combine

sample_tag = 'K562PE4K_HTS'

list_chem = [
    'Imatinib', 
    'Nilotinib', 
    'Bosutinib', 
    'Dasatinib', 
    'Ponatinib', 
    'Asciminib', 
]

# Merge all exons for each replicate and chemical
for chem in list_chem:
    for rep in [1, 2]:
        print(f'Combine: {sample_tag}-{chem}-Replicate{rep}')
        files = glob(f'data/statistics/Filtered_K562PE4K_HTS_Exon*_Rep{rep}_{chem}.csv')
        df_merged = combine_data(files)
        df_merged.to_csv(f'data/statistics/Filtered_{sample_tag}_AllExons_Rep{rep}_{chem}.csv', index=False)

Combine: K562PE4K_HTS-Imatinib-Replicate1
Combine: K562PE4K_HTS-Imatinib-Replicate2
Combine: K562PE4K_HTS-Nilotinib-Replicate1
Combine: K562PE4K_HTS-Nilotinib-Replicate2
Combine: K562PE4K_HTS-Bosutinib-Replicate1
Combine: K562PE4K_HTS-Bosutinib-Replicate2
Combine: K562PE4K_HTS-Dasatinib-Replicate1
Combine: K562PE4K_HTS-Dasatinib-Replicate2
Combine: K562PE4K_HTS-Ponatinib-Replicate1
Combine: K562PE4K_HTS-Ponatinib-Replicate2
Combine: K562PE4K_HTS-Asciminib-Replicate1
Combine: K562PE4K_HTS-Asciminib-Replicate2


In [5]:
# Score calculation: Adjusted LFC / Resistance score
score = VariantScore()
 
for chem in list_chem:

    rep_1 = f'data/statistics/Filtered_K562PE4K_HTS_AllExons_Rep1_{chem}.csv'
    rep_2 = f'data/statistics/Filtered_K562PE4K_HTS_AllExons_Rep2_{chem}.csv'

    adjus_LFC = score.calculate(rep_1, rep_2, var_type='SNV')
    res_score = score.calculate(rep_1, rep_2, var_type='AA')

    adjus_LFC.to_csv(f'data/adjusted_LFC/AdjustedLFC_K562PE4K_{chem}.csv')
    res_score.to_csv(f'data/resistance_score/ResistanceScore_K562PE4K_{chem}.csv')

